In [17]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import numpy as np
import csv
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc 

In [18]:
#
# Mở dụng Webdriver - Modzilla Firefox
#
def open_web_driver():
    options = uc.ChromeOptions() 
    options.headless = False  # Set headless to False to run in non-headless mode

    driver = uc.Chrome(use_subprocess=True, options=options) 
    return driver
 #
 # Đóng web driver
 #
def close_web_driver(driver):
    driver.close()

In [19]:
# Khởi tạo mảng để lưu dữ liệu từ file CSV
all_links = []

# Mở file CSV và đọc dữ liệu từ file
with open("all_links.csv", 'r', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        # row là một list chứa các giá trị từ mỗi hàng trong file CSV
        value = row[0]

        # Thêm giá trị vào mảng
        all_links.append(value)

all_links

['https://batdongsan.com.vn/ban-can-ho-chung-cu-xa-duong-xa-prj-vinhomes-ocean-park-gia-lam/thanh-trung-group-quy-chinh-chu-gui-ban-phap-ly-ro-rang-p-ly-nhat-pr39722407',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-phuong-tay-mo-prj-lumiere-evergreen/-toa-a2-atmos-4pn-140m2-gia-tu-9-5-ty-chiet-khau-16-cho-khach-booking-pr39745574',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-tran-phu-phuong-van-quan-prj-grand-sunlake/doc-quyen-quy-3n-nhinh-3ty-98m2-vao-truoc-700tr-ki-hop-dong-ban-cong-dn-view-ho-pr39736572',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-phuong-tay-mo-prj-imperia-sola-park/co-i-cuoi-cung-so-huu-gia-re-nhat-smart-city-2-gia-tu-5x-trieu-m2-full-noi-that-pr39715399',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-tran-phu-phuong-van-quan-prj-grand-sunlake/-ban-gap-2pn-va-3pn-tai-grang-ha-dong-gia-re-hon-thi-truong-3tr-m2-pr39344303',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-phuong-tay-mo-prj-lumi-hanoi/doc-quyen-quy-dep-tu-cdt-cung-chinh-sach-

In [20]:
all_data = []

In [21]:
#
# Bắt đầu crawl nội dung từng link
#
driver = open_web_driver()

for i in range (1999):

    # Parse HTML
    driver.get(all_links[i])

    time.sleep(2)

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # Tìm tất cả các mục trong thẻ div có class là "re__pr-specs-content-item"
    items = soup.find_all('div', class_='re__pr-specs-content-item')

    data_dict = {}


    # crawl thông tin nhà 
    for item in items:
        title = item.find(class_='re__pr-specs-content-item-title').text.strip()
        value = item.find(class_='re__pr-specs-content-item-value').text.strip()
        data_dict[title] = [value]

    # crawl địa chỉ và ngày đăng
    district = soup.find('a', class_='re__link-se', level=2)
    data_dict["Quận"] = [district.text.strip()]
    city = soup.find('a', class_='re__link-se', level=3)
    data_dict["Thành phố"] = [city.text.strip()]
    category = soup.find('a', class_='re__link-se', level=4)
    data_dict["Loại"] = [category.text.strip()]
    address = soup.find('span', class_='re__pr-short-description js__pr-address')
    data_dict["Địa chỉ"] = [address.text.strip()]
    post_date = soup.find_all('div', class_='re__pr-short-info-item js__pr-config-item')[0]
    data_dict["Ngày đăng"] = [post_date.find(class_='value').text.strip()]
        
    all_data.append(data_dict)

close_web_driver(driver)

KeyboardInterrupt: 

In [22]:
# Tạo DataFrame từ mỗi data_dict trong danh sách và lưu vào một danh sách DataFrame
list_of_dataframes  = [pd.DataFrame(data_dict) for data_dict in all_data]

# Nối các DataFrame lại với nhau theo cột
final_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

# Lưu DataFrame vào file CSV
final_dataframe.to_csv('final_data.csv', index=False)

print("Dữ liệu đã được lưu vào file CSV thành công.")

Dữ liệu đã được lưu vào file CSV thành công.


In [23]:
# Đọc dữ liệu từ file CSV và tạo DataFrame
data = pd.read_csv('final_data.csv', encoding='utf-8')

# Hiển thị DataFrame
data

,Diện tích,Mức giá,Hướng nhà,Hướng ban công,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Quận,Thành phố,Loại,Địa chỉ,Ngày đăng
0,55 m²,"2,07 tỷ",Đông - Bắc,Tây - Nam,2 phòng,1 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ,Hà Nội,Gia Lâm,Căn hộ chung cư tại Vinhomes Ocean Park Gia Lâm,"Dự án Vinhomes Ocean Park Gia Lâm, Xã Dương Xá...",30/04/2024
1,140 m²,"9,5 tỷ",NaN,NaN,4 phòng,3 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ,Hà Nội,Nam Từ Liêm,Căn hộ chung cư tại Lumiere EverGreen,"Dự án Lumiere EverGreen Smart City, Phường Tây...",04/05/2024
2,98 m²,"3,19 tỷ",Đông - Nam,Đông - Nam,3 phòng,2 phòng,Hợp đồng mua bán,Đầy đủ,Hà Nội,Hà Đông,Căn hộ chung cư tại Grand Sunlake,"Dự án Grand Sunlake, Đường Trần Phú, Phường Vă...",03/05/2024
3,59 m²,"2,9 tỷ",Tây - Bắc,Đông - Nam,2 phòng,2 phòng,Hợp đồng mua bán.,Đầy đủ.,Hà Nội,Nam Từ Liêm,Căn hộ chung cư tại Imperia Sola Park,"Dự án Imperia Sola Park, Phường Tây Mỗ, Nam Từ...",06/05/2024
4,92 m²,"3,5 tỷ",NaN,Đông - Nam,2 phòng,2 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ,Hà Nội,Hà Đông,Căn hộ chung cư tại Grand Sunlake,"Dự án Grand Sunlake, Đường Trần Phú, Phường Vă...",05/05/2024
5,81 m²,"5,9 tỷ",Đông - Nam,Đông - Nam,3 phòng,2 phòng,NaN,"Trần, tường, sàn, thiết bị WC cao cấp, điều hò...",Hà Nội,Nam Từ Liêm,Căn hộ chung cư tại Lumi Hanoi,"Dự án Lumi Hanoi, Phường Tây Mỗ, Nam Từ Liêm, ...",06/05/2024
6,75 m²,"4,5 tỷ",Bắc,Nam,2 phòng,2 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ,Hà Nội,Hai Bà Trưng,Căn hộ chung cư tại Times City,"Dự án Times City, Đường Minh Khai, Phường Vĩn...",03/05/2024
7,42 m²,"2,6 tỷ",NaN,NaN,2 phòng,2 phòng,Sổ đỏ/ Sổ hồng,Cơ bản,Hà Nội,Nam Từ Liêm,Căn hộ chung cư tại Masteri West Heights,"Dự án Masteri West Heights, Phường Tây Mỗ, Nam...",06/05/2024
8,55 m²,Thỏa thuận,NaN,NaN,2 phòng,NaN,NaN,NaN,Hà Nội,Nam Từ Liêm,Căn hộ chung cư tại Imperia Sola Park,"Dự án Imperia Sola Park, Phường Tây Mỗ, Nam Từ...",06/05/2024
9,59 m²,"1,85 tỷ",NaN,NaN,2 phòng,1 phòng,Sổ đỏ/ Sổ hồng.,NaN,Hà Nội,Gia Lâm,Căn hộ chung cư tại Vinhomes Ocean Park Gia Lâm,"Dự án Vinhomes Ocean Park Gia Lâm, Xã Dương Xá...",04/05/2024
